In [84]:
class PETs_eval_anonymeter():
    import PETs_Tool

    def __init__(self
                ,data_ori ,data_syn ,data_control
                ,params={} ,**kwargs):
        ####### ####### #######
        # init - params       #
        ####### ####### #######
        # {'evaluate_params' : {'anonymeter':
        default_params = {'SinglingOut' : {'univariate'   : 'Y'
                                          ,'multivariate' : 'N'
                                          }
                         ,'SinglingOut_params' : {'univariate'   : {'n_attacks' : 500}
                                                 ,'multivariate' : {'n_attacks' : 500
                                                                   ,'n_cols'    : 2
                                                                   }
                                                 }
                         ,'Linkability' : 'Y'
                         ,'Linkability_params' : {'n_attacks'   : 2000
                                                 ,'aux_cols'    : []
                                                 # [TODO] study how to set n_neighbors
                                                 ,'n_neighbors' : 10
                                                 # n_jobs follow joblib convention.
                                                 # -1 = all cores,
                                                 # -2 = all execept one
                                                 ,'n_jobs'      : -2
                                                 }
                         ,'Inference' : 'Y'
                         ,'Inference_params' : {'n_attacks'   : 1000
                                               ,'secret'      : ''
                                               # n_jobs follow joblib convention.
                                               # -1 = all cores,
                                               # -2 = all execept one
                                               ,'n_jobs'      : -2
                                               }
                         }
        self.params = PETs_Tool.PETs_util.update_append_nested(default_params ,params)
        self.data_ori     = data_ori
        self.data_syn     = data_syn
        self.data_control = data_control
        self.evaluators   = {}
        self.results      = {}

        import warnings
        with warnings.catch_warnings():
            # anonymeter\evaluators\singling_out_evaluator.py:97:
            # FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.
            # Use isinstance(dtype, CategoricalDtype) instead elif is_categorical_dtype(values)
            warnings.simplefilter("ignore" ,category=FutureWarning)
            # c:\Program Files\Python310\lib\site-packages\anonymeter\stats\confidence.py:215:
            # UserWarning: Attack is as good or worse as baseline model.
            # Estimated rates: attack = 0.30674239114619767, baseline = 0.30773856438771213.
            # Analysis results cannot be trusted. self._sanity_check()
            warnings.simplefilter("ignore" ,category=UserWarning)

            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if any(v == 'Y' for v in self.params['SinglingOut'].values()):
                self.evaluators['SinglingOut'] = {}
                self.results[   'SinglingOut'] = {}

                if self.params['SinglingOut']['univariate'] == 'Y':
                    self.__SinglingOutEvaluator(mode = 'univariate'
                                               ,**self.params['SinglingOut_params']['univariate']
                                               )
                # if self.params['SinglingOut']['multivariate'] == 'Y':
                #     self.__SinglingOutEvaluator(mode = 'multivariate'
                #                                ,**self.params['SinglingOut_params']['multivariate']
                #                                )
            ####### ####### #######
            # init - SinglingOut  #
            ####### ####### #######
            if self.params['Linkability'] == 'Y':
                self.__LinkabilityEvaluator(**self.params['Linkability_params'])


            if self.params['Inference'] == 'Y':
                self.evaluators['Inference'] = {}
                self.results[   'Inference'] = {}
                if self.params['Inference_params']['secret'] == '':
                    for __secret in self.data_syn.columns:
                        self.params['Inference_params']['secret'] = __secret
                        self.__InferenceEvaluator(**self.params['Inference_params'])
                else:
                    self.__InferenceEvaluator(**self.params['Inference_params'])

            



    ####### ####### #######
    # SinglingOut         #
    ####### ####### #######
    def __SinglingOutEvaluator(self ,mode ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Singling-Out Evaluator')
        from anonymeter import evaluators
        __evaluator = evaluators.SinglingOutEvaluator(ori       = self.data_ori
                                                     ,syn       = self.data_syn
                                                     ,control   = self.data_control
                                                     ,**kwargs
                                                     )
        try:
            __evaluator.evaluate(mode=mode)
            self.evaluators['SinglingOut'][mode] = __evaluator
            self.results[   'SinglingOut'][mode] = self.__Result(self.evaluators['SinglingOut'][mode])
        except RuntimeError as ex:
            print(f"Singling out evaluation failed with {ex}."
                   "Please re-run this cell."
                   "For more stable results increase `n_attacks`. Note that this will "
                   "make the evaluation slower.")
        # multivariance
        # try:
        #     __evaluator.evaluate(mode='multivariate')
        #     __risk = __evaluator.risk()
        #     print(__risk)
        #     res = __evaluator.results()
        #     print(__evaluator.queries()[:3])
        # except RuntimeError as ex:
        #     print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
        #           "For more stable results increase `n_attacks`. Note that this will "
        #           "make the evaluation slower.")
        print(f"Singling-Out Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Linkability         #
    ####### ####### #######
    def __LinkabilityEvaluator(self ,n_jobs ,aux_cols ,**kwargs):
        import time
        __time_start = time.time()
        print('Now is Linkability Evaluator')
        from anonymeter import evaluators
        __str_aux_cols = "\nand ".join(f"[{', '.join(row)}]" for row in aux_cols)
        print(f"aux_cols are {__str_aux_cols}.")
        __evaluator = evaluators.LinkabilityEvaluator(ori      = self.data_ori
                                                     ,syn      = self.data_syn
                                                     ,control  = self.data_control
                                                     ,aux_cols = aux_cols
                                                     ,**kwargs
                                                     )
        __evaluator.evaluate(n_jobs=n_jobs)
        self.evaluators['Linkability'] = __evaluator
        self.results[   'Linkability'] = self.__Result(self.evaluators['Linkability'])
        print(f"Linkability Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Inference           #
    ####### ####### #######
    def __InferenceEvaluator(self ,secret ,n_jobs ,**kwargs):
        import time
        __time_start = time.time()
        print(f"Now is Inference Evaluator: secret is {secret}.")
        from anonymeter import evaluators
        __aux_cols = [col for col in self.data_syn.columns if col != secret]
        __evaluator = evaluators.InferenceEvaluator(ori      = self.data_ori
                                                   ,syn      = self.data_syn
                                                   ,control  = self.data_control
                                                   ,aux_cols = __aux_cols
                                                   ,secret   = secret
                                                   ,**kwargs
                                                   )
        __evaluator.evaluate(n_jobs=n_jobs)
        self.evaluators['Inference'][secret] = __evaluator
        self.results[   'Inference'][secret] = self.__Result(self.evaluators['Inference'][secret])
        print(f"Inference Evaluator time: {round(time.time()-__time_start ,4)} sec.")



    ####### ####### #######
    # Result              #
    ####### ####### #######
    def __Result(self ,__evaluator): 
        import numpy as np
        __dict_result    = {}
        __para_to_handle = [('Risk'              , ['risk()'    ,'value'                 ])
                           ,('Risk_CI_btm'       , ['risk()'    ,'ci[0]'                 ])
                           ,('Risk_CI_top'       , ['risk()'    ,'ci[1]'                 ])
                           ,('Attack_Rate'       , ['results()' ,'attack_rate'   ,'value'])
                           ,('Attack_Rate_err'   , ['results()' ,'attack_rate'   ,'error'])
                           ,('Baseline_Rate'     , ['results()' ,'baseline_rate' ,'value'])
                           ,('Baseline_Rate_err' , ['results()' ,'baseline_rate' ,'error'])
                           ,('Control_Rate'      , ['results()' ,'control_rate'  ,'value'])
                           ,('Control_Rate_err'  , ['results()' ,'control_rate'  ,'error'])
                           ]
        for __key ,__attrs in __para_to_handle:
            try:
                __attr_value = __evaluator
                for __attr in __attrs:
                    if '()' in __attr:
                        __method_name = __attr.split('(')[0]
                        if hasattr(__attr_value ,__method_name):
                            __method = getattr(__attr_value ,__method_name)
                            if callable(__method):
                                __attr_value = __method()
                            else:
                                __dict_result[__key] = np.nan
                                break
                        else:
                            __dict_result[__key] = np.nan
                            break
                    elif '[' in __attr:
                        __attr_name = __attr.split('[')[0]
                        __index = int(__attr.split('[')[1].rstrip(']'))
                        if hasattr(__attr_value, __attr_name)\
                        and isinstance(getattr(__attr_value, __attr_name) ,(list, dict, tuple)):
                            try:
                                __attr_value = getattr(__attr_value  ,__attr_name)[__index]
                            except (IndexError, KeyError):
                                __dict_result[__key] = np.nan
                                break
                        else:
                            __dict_result[__key] = np.nan
                            break
                    else:
                        __attr_value = getattr(__attr_value, __attr)
                __dict_result[__key] = __attr_value
            except Exception as e:
                __dict_result[__key] = np.nan
        return __dict_result

In [90]:
import os
import PETs_Tool


folder_raw = '.\\data'
folder_SD  = '.\\data_dpsd'



dict_PETs_setting = {'adult' : {'filename'    : '[Adult]'
                               ,'read_params' : {'downcast'       : 'N' # 'Y'
                                                ,'label_encoding' : 'Y'}
                            #    ,'describe_params' : {'data': {'?'}}
                               }
                    }
dict_filename = {'adult': {'raw' : '[Adt Income] adult.csv'
                          }
                }
dict_filename = {key: {**value, 'params': dict_PETs_setting.get(key, {})} 
                 for key, value in dict_filename.items()}
# print(dict_filename)



# update default in SDV
list_adult_SD_SDV_train = ['GaussianCoupula'
                        #    'CoupulaGAN'
                        #   ,'CTGAN'
                        #   ,'GaussianCoupula'
                        #   ,'TVAE'
                          ]
dict_adult_SD_SDV_train = {k: {'sd_params': {'model': k}} for k in list_adult_SD_SDV_train}
dict_filename = {key: PETs_Tool.PETs_util.update_append_nested(value
                                                              ,{'SD_train': dict_adult_SD_SDV_train}
                                                              ) 
                 for key, value in dict_filename.items()}
dict_adult_SD_SDV_train_default = {'sd_params': {'save_model': 'N'}
                                  ,'sample'   : 'Y'
                                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                                     ,'save_data' : 'N'}
                                  }
dict_adult_SD_train = {'SDV' : {key: PETs_Tool.PETs_util.update_append_nested(dict_adult_SD_SDV_train_default
                                                                       ,dict_adult_SD_SDV_train.get(key ,{})
                                                                       )
                                for key in dict_adult_SD_SDV_train
                               }
                      }
dict_filename = {k1: {k2: dict_adult_SD_train if k2 == 'SD_train' else v2
                      for k2 ,v2 in dict_filename[k1].items()
                     } 
                 for k1 in dict_filename
                }
# print(dict_filename)


same_sample_times = 5
dict_bootstrap = {'bootstrap_params' : {'bootstrap_time' : 5 # 3 # 30
                                       ,'sample_ratio'   : 0.8
                                       ,'random_state'   : None
                                       }
                 ,'index_params' : {'index_save'      : 'N'
                                   ,'index_save_data' : 'N'
                                   ,'index_filename'  : 'Anomymeter'
                                   }
                 }
# import json
# print(json.dumps(dict_bootstrap ,indent=4))

__Synther_param = {'sd_params': {'metadata' : 'SingleTable'
                                ,'save_model' : 'N'}
                  ,'sample' : 'Y'
                  ,'sample_params' : {'sample_rows_as_raw' : 'Y'
                                     ,'save_data' : 'N'}
                  }
aux_cols = [['age' ,'fnlwgt' ,'race' ,'gender' ,'native-country']
           ,['workclass' ,'education' ,'capital-gain' ,'capital-loss' ,'hours-per-week']
           ]

__param_anonymeter = {'SinglingOut': {'univariate'   : 'Y'
                                     ,'multivariate' : 'N'
                                     }
                     ,'Linkability' : 'Y'
                     ,'Linkability_params' : {'aux_cols' : aux_cols}
                     ,'Inference' : 'Y'
                     ,'Inference_params' : {'secret' : ''} # all columns
                     }
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for key, value in d.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_dict(value, new_key, sep=sep).items())
        else:
            items.append((new_key, value))
    return dict(items)






__dict_anonymeter_result = {}
for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __Loader = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    )
    __df_ori = __Loader.data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    dict_boostrap_index = PETs_Tool.PETs_util.df_bootstrap(__df_ori ,dict_bootstrap)
    digits_max_boostrap = len(str(max(dict_boostrap_index.keys()))) # max boostrap times for fill zero
    digits_same_sample  = len(str(    same_sample_times          )) # same sample times for fill zero         

    for __library ,__dict_sd in v['SD_train'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__params in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")

            import copy
            for time ,dict_idx in dict_boostrap_index.items():
                __trail_name = f"{__filename}_{__library}_{__method}_{str(time+1).zfill(digits_max_boostrap)}"
                print(f"Trail as {__trail_name}.")

                ####### ####### #######
                # init - ori ,ctrl ,syn #
                ####### ####### #######
                __Synther = copy.deepcopy(__Loader)
                __data = __Synther.data.copy()
                __Synther.data_ori     = __data.loc[dict_boostrap_index[time]['idx_train'     ]].reset_index(drop=True)
                __Synther.data_control = __data.loc[dict_boostrap_index[time]['idx_validation']].reset_index(drop=True)
                __Synther.data         = __Synther.data_ori.copy()
                __Synther.data_syn     = PETs_Tool.PETs_SD_SDV(__Synther ,__Synther_param).synthetic_data
                __Synther.data = __data.copy()

                for same_sample_time in range(same_sample_times):
                    __trail_name = f"{__trail_name}_{str(same_sample_time+1).zfill(digits_same_sample)}"
                    print(f"Trail as {__trail_name}.")
                    ####### ####### #######
                    # _anonymeter         #
                    ####### ####### #######
                    __anonymeter = PETs_eval_anonymeter(data_ori     = __Synther.data_ori
                                                    ,data_syn     = __Synther.data_syn
                                                    ,data_control = __Synther.data_control
                                                    ,params = __param_anonymeter
                                                    )
                    __evaluator = __anonymeter.evaluators
                    
                    __dict_anonymeter_result[__trail_name] = {}
                    for __result_key ,__result_value in __anonymeter.results.items():
                        __dict_anonymeter_result[__trail_name].update(flatten_dict(__result_value ,parent_key=__result_key))

                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets



Now is [Adult]: [Adt Income] adult.csv comparision.
Now is [Adult] original data been load: [Adt Income] adult.csv.
Bootstrap df: 5 times is done.
Now is [Adult] by library SDV.
Now is [Adult] by method GaussianCoupula in library SDV.
Trail as [Adult]_SDV_GaussianCoupula_1.
Metafile loading time: 0.029 sec.
We are execute SingleTable - GaussianCoupula.
Model training time: 20.6221 sec.
Sample as same as raw data: # 48842 rows data by GaussianCoupula in 2.6976 秒


TypeError: 'int' object is not iterable

In [258]:
import pandas as pd
__df_result = pd.DataFrame.from_dict(__dict_anonymeter_result ,orient='index')
# [TODO] 按照 SinglingOut/Linkability/Inference 以及 colname 排序
__df_result.to_csv('Justyn_anonymeter_20231128_1301.csv')

- 這是已經有檔案的做法 (Unfinished)

In [ ]:
dict_adult_SD_SDV = {'CoupulaGAN'      : ['[Adt Income]_[SDV]_[CoupulaGAN]_20231106_113038_Asia_Taipei_10000.csv']
                    ,'CTGAN'           : ['[Adt Income]_[SDV]_[CTGAN]_20231106_093702_Asia_Taipei_10000.csv']
                    ,'GaussianCoupula' : ['[Adt Income]_[SDV]_[GaussianCoupula]_20231106_093622_Asia_Taipei_10000.csv']
                    ,'TVAE'            : ['[Adt Income]_[SDV]_[TVAE]_20231106_105252_Asia_Taipei_10000.csv']
                    }
dict_filename = {key: {**value ,'SDV': dict_adult_SD_SDV} 
                 for key, value in dict_filename.items()}



for __dataset ,v in dict_filename.items():
    __filename = v['params']['filename']
    print(f"Now is {__filename}: {v['raw']} comparision.")

    __df_ori = PETs_Tool.PETs_Loader(os.path.join(folder_raw ,v['raw'])
                                    ,v['params']
                                    ).data
    print(f"Now is {__filename} original data been load: {v['raw']}.")

    for __library ,__dict_sd in v['SD'].items():
        print(f"Now is {__filename} by library {__library}.")

        for __method ,__list_sd in __dict_sd.items():
            print(f"Now is {__filename} by method {__method} in library {__library}.")
            __list_sd = [__list_sd] if isinstance(__list_sd ,str) else __list_sd
            for __filename_sd in __list_sd:
                # 20231116, Justyn: 這裡是合成資料有落地
                #                   如果必要，可以直接調用 PETs_SD_* 結果的.synthetic_data
                __df_syn = PETs_Tool.PETs_Loader(os.path.join(folder_SD ,__filename_sd)
                                                ,v['params']
                                                ).data
                print(f"Now is {__filename} synthetic data been load: {__filename_sd}.")

                # 20231116, Justyn: refer to
                # https://github.com/statice/anonymeter/blob/main/notebooks/anonymeter_example.ipynb
                # https://storage.googleapis.com/statice-public/anonymeter-datasets

